# Gaussova eliminacija
---

## Općenito

Sustav $Ax=b$
se rješava u tri koraka (__bez pivotiranja__):

1. $A=LU$ (LU rastav, $O(\frac{2}{3}n^3)$ operacija),
2. $Ly=b$ (donje trokutrasti sustav, $n^2$ operacija),
3. $Ux=y$ (gornje torkutasti sustav, $n^2$ operacija).

Ukoliko pivotiramo, tada je

1. $PA=LU$, 
2. $Ly=P^T b$,
3. $Ux=y$. 

## LU rastav

In [1]:
function mylu{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    U=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n]) # This acccepts blocks and numbers
    L=map(Float64,[zero(A[1,1]) for i=1:n, j=1:n])
    for k=1:n
        L[k,k]=one(A[1,1])
        for i=k+1:n
            L[i,k]=A[i,k]/A[k,k]
            for j=k+1:n
                A[i,j]=A[i,j]-L[i,k]*A[k,j]
            end
        end
        for j=k:n
            U[k,j]=A[k,j]
        end
    end
    L,U
end

mylu (generic function with 1 method)

In [2]:
A=rand(6,6)
# A=[2.0 2;3 4]

6×6 Array{Float64,2}:
 0.820503  0.524823  0.742737   0.70906   0.458562  0.44023 
 0.833192  0.66689   0.478398   0.479932  0.928422  0.742974
 0.412929  0.341976  0.485852   0.235312  0.444012  0.89146 
 0.651897  0.778236  0.762346   0.524772  0.82272   0.908214
 0.849603  0.182454  0.0586779  0.106927  0.222613  0.644869
 0.484679  0.352626  0.170392   0.995743  0.933397  0.548205

In [3]:
L,U=mylu(A)

([1.0 0.0 … 0.0 0.0; 1.01547 1.0 … 0.0 0.0; … ; 1.03547 -2.69491 … 1.0 0.0; 0.59071 0.318086 … -2.0268 1.0], [0.820503 0.524823 … 0.458562 0.44023; 0.0 0.13395 … 0.462768 0.295936; … ; 0.0 0.0 … -0.596473 -0.469174; 0.0 0.0 … 0.0 1.89561])

In [4]:
L

6×6 Array{Float64,2}:
 1.0        0.0        0.0        0.0       0.0     0.0
 1.01547    1.0        0.0        0.0       0.0     0.0
 0.503264   0.5812     1.0        0.0       0.0     0.0
 0.794509   2.69697    3.36355    1.0       0.0     0.0
 1.03547   -2.69491   -5.33733   -2.14854   1.0     0.0
 0.59071    0.318086  -0.663118   1.21307  -2.0268  1.0

In [5]:
U

6×6 Array{Float64,2}:
 0.820503  0.524823   0.742737   0.70906     0.458562    0.44023 
 0.0       0.13395   -0.275826  -0.240094    0.462768    0.295936
 0.0       0.0        0.27237    0.0180098  -0.0557272   0.49791 
 0.0       0.0        0.0        0.548367   -0.602244   -1.91443 
 0.0       0.0        0.0        0.0        -0.596473   -0.469174
 0.0       0.0        0.0        0.0         0.0         1.89561 

In [6]:
L*U-A

6×6 Array{Float64,2}:
  0.0          0.0   0.0           0.0          0.0           0.0        
 -1.11022e-16  0.0   0.0           0.0          0.0           0.0        
  5.55112e-17  0.0   0.0           0.0          0.0           0.0        
  0.0          0.0   0.0           1.11022e-16  0.0           0.0        
  0.0          0.0  -6.245e-17    -6.93889e-17  1.11022e-16   0.0        
  0.0          0.0  -2.77556e-17   0.0          0.0          -2.22045e-16

## Trokutasti sustavi

In [7]:
function myU{T}(U::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=n:-1:1
       for j=n:-1:i+1
            b[i]=b[i]-U[i,j]*b[j]
       end
        b[i]=b[i]/U[i,i]
    end
    b
end

function myL{T}(L::Array{T},b1::Array{T})
    b=deepcopy(b1)
    n=length(b)
    for i=1:n
        for j=1:i-1
            b[i]=b[i]-L[i,j]*b[j]
        end
        b[i]=b[i]/L[i,i]
    end
    b
end

myL (generic function with 1 method)

In [8]:
b=rand(6)

6-element Array{Float64,1}:
 0.713561
 0.27599 
 0.952593
 0.639983
 0.373263
 0.567357

In [9]:
# Rijesimo sustav pomocu ugradjene funkcije:
x=A\b

6-element Array{Float64,1}:
 -0.073612 
 -1.80486  
  1.24633  
  0.540508 
 -0.0674805
  1.00672  

In [10]:
# Rijesimo sustav pomocu nasih funkcija
y=myL(L,b)

6-element Array{Float64,1}:
  0.713561
 -0.448607
  0.854214
 -1.59026 
 -0.432077
  1.90836 

In [11]:
x1=myU(U,y)

6-element Array{Float64,1}:
 -0.073612 
 -1.80486  
  1.24633  
  0.540508 
 -0.0674805
  1.00672  

In [12]:
# Usporedimo rjesenja
x-x1

6-element Array{Float64,1}:
 -3.88578e-16
 -4.44089e-16
  1.55431e-15
 -9.99201e-16
  7.77156e-16
 -6.66134e-16

## Brzina

Program `mylu()` je spor. Između ostalog, alocira nepotrebno tri matrice i ne računa s blok matricama.

Program se može preformulirati na načun da su i $L$ i $U$ spremljene u polje $A$, pri čemu se dijagonala od $L$ ne sprema jer su svi elementi jednaki 1 (vidi [Introduction to Linear Algebra, str. 100][St09]):

[St09]: https://books.google.hr/books?id=M19gPgAACAAJ&dq=strang%20introduction&hl=hr&source=gbs_book_other_versions "Gilbert Strang, 'Introduction to Linear Algebra, 4th Edition', Wellesley-Cambridge Press, 2009"


In [13]:
function mylu1{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        rho=k+1:n
        A[rho,k]=A[rho,k]/A[k,k]
        A[rho,rho]=A[rho,rho]-A[rho,k]*A[k,rho]'
    end
    A
end

mylu1 (generic function with 1 method)

In [14]:
mylu1(A)

6×6 Array{Float64,2}:
 0.820503   0.524823   0.742737   0.70906     0.458562    0.44023 
 1.01547    0.13395   -0.275826  -0.240094    0.462768    0.295936
 0.503264   0.5812     0.27237    0.0180098  -0.0557272   0.49791 
 0.794509   2.69697    3.36355    0.548367   -0.602244   -1.91443 
 1.03547   -2.69491   -5.33733   -2.14854    -0.596473   -0.469174
 0.59071    0.318086  -0.663118   1.21307    -2.0268      1.89561 

In [15]:
L

6×6 Array{Float64,2}:
 1.0        0.0        0.0        0.0       0.0     0.0
 1.01547    1.0        0.0        0.0       0.0     0.0
 0.503264   0.5812     1.0        0.0       0.0     0.0
 0.794509   2.69697    3.36355    1.0       0.0     0.0
 1.03547   -2.69491   -5.33733   -2.14854   1.0     0.0
 0.59071    0.318086  -0.663118   1.21307  -2.0268  1.0

In [16]:
U

6×6 Array{Float64,2}:
 0.820503  0.524823   0.742737   0.70906     0.458562    0.44023 
 0.0       0.13395   -0.275826  -0.240094    0.462768    0.295936
 0.0       0.0        0.27237    0.0180098  -0.0557272   0.49791 
 0.0       0.0        0.0        0.548367   -0.602244   -1.91443 
 0.0       0.0        0.0        0.0        -0.596473   -0.469174
 0.0       0.0        0.0        0.0         0.0         1.89561 

Usporedimo brzine LAPACK-ovog programa `lu()` i našeg naivnog programa `mylu()`na većoj dimenziji. 

Izvedite program par puta radi točnijeg mjerenja brzine.

In [17]:
n=512
A=rand(n,n)

512×512 Array{Float64,2}:
 0.40525    0.474194   0.740019   …  0.954611   0.451167    0.604163 
 0.843135   0.667057   0.712185      0.343417   0.832981    0.71743  
 0.341065   0.269558   0.853296      0.214809   0.29484     0.543447 
 0.105764   0.0864902  0.368742      0.218207   0.438381    0.889953 
 0.0291881  0.765378   0.144996      0.721853   0.596918    0.567829 
 0.717623   0.252159   0.852092   …  0.657859   0.368521    0.329335 
 0.151418   0.784974   0.0680676     0.158073   0.00789914  0.253514 
 0.692507   0.555201   0.639499      0.0826382  0.0811569   0.230314 
 0.67224    0.567081   0.948226      0.528137   0.0366534   0.483208 
 0.33151    0.0743544  0.034459      0.806051   0.112924    0.526945 
 0.249748   0.593291   0.615969   …  0.153594   0.933527    0.188701 
 0.697734   0.490018   0.590119      0.419692   0.0566826   0.14139  
 0.721466   0.0922171  0.536239      0.388559   0.195488    0.932501 
 ⋮                                ⋱             ⋮               

In [19]:
@time lu(A);

  0.070555 seconds (21 allocations: 6.009 MiB)


In [21]:
@time mylu1(A);

  0.755235 seconds (9.07 k allocations: 1.003 GiB, 18.06% gc time)


### Blok varijanta

`mylu()` je nekoliko desetaka puta sporiji od `lu()`. `mylu1()` je malo sporiji od `lu()`.

Probajmo s blokovima:

Preradimo `mylu1()` za rad s blokovima (nemamo ugrađeno pivotiranje!)

In [22]:
function mylu2{T}(A1::Array{T}) # Strang, page 100
    A=deepcopy(A1)
    n,m=size(A)
    for k=1:n-1
        for rho=k+1:n
            A[rho,k]=A[rho,k]/A[k,k]
            for l=k+1:n
                A[rho,l]=A[rho,l]-A[rho,k]*A[k,l]
            end
        end
    end
    A
end

mylu2 (generic function with 1 method)

Napravimo prvo mali test:

In [23]:
# Probajte k,l=32,16 i k,l=64,8
k,l=2,4
Ab=[rand(k,k) for i=1:l, j=1:l];

In [24]:
A0=mylu2(Ab)

4×4 Array{Array{Float64,2},2}:
 [0.261794 0.275895; 0.380011 0.269714]  …  [0.544035 0.960539; 0.0104482 0.376034]  
 [-5.58404 6.07161; -1.02141 3.2697]        [3.25825 3.60804; 0.716145 0.564049]     
 [3.99366 -2.15624; -0.39357 0.595157]      [-0.498951 -1.45724; -0.570471 -0.454331]
 [-5.41876 5.8063; 1.94156 -0.0137676]      [-7.70252 -25.9912; -1.07398 -1.90978]   

In [25]:
# Provjera
U=triu(A0)
L=tril(A0)
for i=1:maximum(size(L))
    L[i,i]=eye(L[1,1])
end

In [26]:
Res=L*U-Ab

4×4 Array{Array{Float64,2},2}:
 [0.0 0.0; 0.0 0.0]                           …  [0.0 0.0; 0.0 0.0]                         
 [4.44089e-16 2.22045e-16; -1.11022e-16 0.0]     [2.22045e-16 -2.22045e-16; 0.0 0.0]        
 [0.0 0.0; 0.0 0.0]                              [-1.11022e-16 0.0; 0.0 1.11022e-16]        
 [0.0 0.0; -1.11022e-16 0.0]                     [1.11022e-15 -2.22045e-16; 0.0 2.22045e-16]

In [27]:
# pretvaranje blok matrice u obicnu
unblock(A) = mapreduce(identity, hcat, [mapreduce(identity, vcat, A[:,i]) 
        for i = 1:size(A,2)])

unblock (generic function with 1 method)

In [28]:
norm(unblock(Res))

1.1964539659109401e-15

Sada probajmo veću dimenziju:

In [29]:
# Probajmo vece dimenzije (n=k*l)
k,l=32,16
Ab=[rand(k,k) for i=1:l, j=1:l];

In [31]:
@time mylu2(Ab);

  0.112305 seconds (3.95 k allocations: 26.553 MiB, 7.53% gc time)


Vidimo da je `mylu2()` gotovo jednako brz kao `lu()`, uz napomenu da `mylu2()` nema ugrađeno pivotiranje. 

## Pivotiranje

Stanardne implementacije uvijek računaju Gaussovu eliminaciju s _parcijalnim pivotiranjem_:

> u svakom koraku se retci pivotiranju tako da pivotni element ima najveću apsolutnu vrijednost u danom stupcu. Na taj 
> način je 
> 
> $$|L_{ij}| \leq 1,$$
> 
> što u praksi dovoljno spriječava rast elemenata.


In [32]:
A=[0.00003 1;2 3]

2×2 Array{Float64,2}:
 3.0e-5  1.0
 2.0     3.0

In [34]:
L,U=mylu(A)
L

2×2 Array{Float64,2}:
     1.0  0.0
 66666.7  1.0

In [35]:
U

2×2 Array{Float64,2}:
 3.0e-5       1.0
 0.0     -66663.7

In [36]:
# s pivoritranjem
P=[0 1;1 0]
L,U=mylu(P*A)
L

2×2 Array{Float64,2}:
 1.0     0.0
 1.5e-5  1.0

In [38]:
U

2×2 Array{Float64,2}:
 2.0  3.0     
 0.0  0.999955

In [39]:
L*U-P*A

2×2 Array{Float64,2}:
 0.0  0.0
 0.0  0.0

In [40]:
# Slučajna matrica, standardna funkcija lu()
A=rand(5,5)
L,U,P=lu(A)

([1.0 0.0 … 0.0 0.0; 0.6238 1.0 … 0.0 0.0; … ; 0.185404 0.480137 … 1.0 0.0; 0.656005 0.0635691 … -0.295977 1.0], [0.949604 0.2934 … 0.827989 0.336356; 0.0 0.673516 … -0.445806 0.237281; … ; 0.0 0.0 … 0.895012 0.923128; 0.0 0.0 … 0.0 0.373798], [3, 1, 4, 5, 2])

In [42]:
L

5×5 Array{Float64,2}:
 1.0        0.0         0.0        0.0       0.0
 0.6238     1.0         0.0        0.0       0.0
 0.580308  -0.191901    1.0        0.0       0.0
 0.185404   0.480137   -0.151097   1.0       0.0
 0.656005   0.0635691  -0.179701  -0.295977  1.0

In [33]:
L*U-A[P,:]

5×5 Array{Float64,2}:
  0.0          0.0           0.0          0.0  0.0
  0.0          0.0           0.0          0.0  0.0
 -2.77556e-17  0.0           0.0          0.0  0.0
 -5.55112e-17  0.0           0.0          0.0  0.0
 -5.55112e-17  2.77556e-17  -5.55112e-17  0.0  0.0

In [43]:
U

5×5 Array{Float64,2}:
 0.949604  0.2934    0.34068    0.827989  0.336356
 0.0       0.673516  0.379273  -0.445806  0.237281
 0.0       0.0       0.265967   0.34123   0.835845
 0.0       0.0       0.0        0.895012  0.923128
 0.0       0.0       0.0        0.0       0.373798

In [44]:
P

5-element Array{Int64,1}:
 3
 1
 4
 5
 2

In [45]:
L*U-A[P,:]

5×5 Array{Float64,2}:
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  0.0           0.0        
 0.0  0.0  0.0  1.11022e-16   1.11022e-16
 0.0  0.0  0.0  0.0          -5.55112e-17

### Potpuno pivotiranje

Sljedeći program računa Gaussovu eliminaciju s _potpunim pivotiranjem_ - u svakom koraku se retci i stupci zamijene na način da se na pivotnu poziciju dovede element koji ima najveću apsolutnu vrijednost u trenutnoj podmatrici.

In [46]:
function gecp{T}(A1::Array{T})
    # Gaussova eliminacija s potpunim pivotiranjem
    # Izlaz: Pr*L*U*Pc'=A ili Pr'*A*Pc=L*U
    A=deepcopy(A1)
    n,m=size(A)
    Pr=eye(n,n)
    Pc=eye(n,n)
    D=zeros(n)
    for i=1:n-1
        am1,im1=findmax(abs.(A[i:n,i:n]),1)
        am,JJ=findmax(am1)
        II=mod(im1[JJ],(n-i+1))
        if II==0
            II=n-i+1
        end
        imax=II+i-1
        jmax=JJ+i-1
        #  zamijena redaka
        if (imax != i)
            temp = Pr[:,i]
            Pr[:,i] = Pr[:,imax]
            Pr[:,imax] = temp
            temp = A[i,:]
            A[i,:] = A[imax,:]
            A[imax,:] = temp
        end
        # zamijena stupaca
        if (jmax != i)
            temp = Pc[:,i]
            Pc[:,i] = Pc[:,jmax]
            Pc[:,jmax] = temp
            temp = A[:,i]
            A[:,i] = A[:,jmax]
            A[:,jmax] = temp
        end
        # eliminacija
        D[i]=A[i,i]
        A[i+1:n,i] = A[i+1:n,i]/D[i]
        A[i+1:n,i+1:n] = A[i+1:n,i+1:n] - A[i+1:n,i]*A[i,i+1:n]'
        A[i,i+1:n]=A[i,i+1:n]/D[i]
    end
    D[n]=A[n,n]
    L=eye(n,n)+tril(A,-1)
    U=eye(n,n)+triu(A,1)
    U=diagm(D)*U
    L,U,Pr,Pc
end

gecp (generic function with 1 method)

In [47]:
n=5
A=rand(n,n)
b=rand(n)

5-element Array{Float64,1}:
 0.895372 
 0.266723 
 0.438844 
 0.179129 
 0.0374415

In [49]:
L,U,Pr,Pc=gecp(A)

([1.0 0.0 … 0.0 0.0; 0.384118 1.0 … 0.0 0.0; … ; 0.994112 0.0337325 … 1.0 0.0; 0.382726 0.375922 … 0.361382 1.0], [0.911643 0.173453 … 0.245796 0.781716; 0.0 0.734821 … 0.420347 -0.259876; … ; 0.0 0.0 … 0.458099 -0.0503244; 0.0 0.0 … 0.0 0.21446], [0.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 1.0; 1.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0])

In [50]:
Pr*L*U*Pc'-A

5×5 Array{Float64,2}:
 0.0  0.0   0.0           0.0          0.0
 0.0  0.0   0.0          -2.77556e-17  0.0
 0.0  0.0   0.0           0.0          0.0
 0.0  0.0   0.0           0.0          0.0
 0.0  0.0  -1.11022e-16  -5.55112e-17  0.0

In [51]:
y=myL(L,Pr'*b)

5-element Array{Float64,1}:
  0.266723 
  0.792919 
 -0.670169 
  0.0482167
 -0.123004 

In [52]:
z=myU(U,y)

5-element Array{Float64,1}:
  1.07001  
  1.29278  
 -1.2779   
  0.0422461
 -0.573554 

In [53]:
x=Pc*z

5-element Array{Float64,1}:
 -0.573554 
  1.07001  
  1.29278  
  0.0422461
 -1.2779   

In [54]:
A*x-b

5-element Array{Float64,1}:
  0.0        
 -1.11022e-16
 -1.11022e-16
 -2.22045e-16
 -1.11022e-16

## Točnost

Neka je zadan sustav $Ax=b$, pri čemu je matrica $A$ regularna.

Da bi primijenili koncepte iz bilježnice [NA04_Pogreska_unatrag_i stabilni_algoritmi](NA04_Pogreska_unatrag_i_stabilni_algoritmi.ipynb), potrebno je:

1. napraviti teoriju smetnje za dani problem
2. analizirati pogreške algoritma (Gaussove eliminacije)

### Teorija smetnje

Neka je 

$$
(A+\delta A)\hat x=(b+\delta b)
$$

za neki $\hat x=x+\delta x$.

Želimo ocijeniti 

$$
\frac{\| \hat x - x \|}{\| x\|} \equiv \frac{\| \delta x\|}{\| x\|}.
$$

Uvedimo oznake (npr. prema [Matrix Computations, poglavlje 2.6.2][GVL13])

$$
\delta A=\varepsilon F, \quad \delta b=\varepsilon f, \qquad \hat x=x(\varepsilon),
$$
čime smo dobili jednodimenzionalni problem 

$$
(A+\varepsilon F)\,x(\varepsilon)=b+\varepsilon f.
$$

za neke (nepoznate) matricu $F$ i vektor $f$. 

Deriviranje po $\varepsilon$ daje

$$
Fx(\varepsilon)+(A+\varepsilon F)\, x(\varepsilon)=f.
$$

Uvrštavanje $\varepsilon=0$ daje

$$
F x+A\dot x(0)=f,
$$

odnosno

$$
\dot x(0)=A^{-1}(f-Fx).
$$

Taylorov razvoj oko $\varepsilon=0$ glasi

$$
x(\varepsilon)=x(0)+\varepsilon \dot x(0) +O(\varepsilon^2),
$$

odnosno, uz zanemarivanje člana $O(\varepsilon^2)$,

$$
\hat x-x=\varepsilon A^{-1}(f-Fx)=A^{-1} (\varepsilon f + \varepsilon F x) = A^{-1} (\delta b + \delta A x).
$$

Svojstva norme povlače

$$
\| \hat x-x\|\leq \| A^{-1} \| (\| \delta b \|  + \| \delta A \| \cdot \|  x\| ).
$$

Konačno, zbog $\| b\| \leq \| A\| \| x\|$, imamo

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \| A\|  \cdot \| A^{-1} \| \bigg(\frac{\| \delta b \|}{\|b\|}  + \frac{\| \delta A \|}{ \|  A\|} \bigg). \tag{1}
$$

Broj 
$$
\kappa(A)\equiv \| A\|  \cdot \| A^{-1} \|
$$ 

je __uvjetovanost__ (__kondicija__)  matrice $A$ i kazuje nam 

> koliko se relativno uvećaju relativne promjene u polaznim podacima (matrici $A$ i vektoru $b$).

Pogledajmo primjer iz [Numeričke matematike, str. 42][RS04]:


[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."

In [55]:
A= [0.234 0.458; 0.383 0.750]

2×2 Array{Float64,2}:
 0.234  0.458
 0.383  0.75 

In [56]:
b=[0.224;0.367]

2-element Array{Float64,1}:
 0.224
 0.367

In [57]:
x=A\b

2-element Array{Float64,1}:
 -1.0
  1.0

In [58]:
δb=[0.00009; 0.000005]
x1=A\(b+δb)

2-element Array{Float64,1}:
 -0.241744
  0.612791

In [59]:
cond(A), norm(δb)/norm(b), norm(x1-x)/norm(x)

(11322.197586092605, 0.0002096449170953002, 0.6020311134825742)

In [60]:
δA=[-0.001 0;0 0]
x2=(A+δA)\b

2-element Array{Float64,1}:
 0.129518
 0.423193

In [61]:
cond(A), norm(δA)/norm(A), norm(x2-x)/norm(x)

(11322.197586092605, 0.0010134105230118603, 0.896804787832142)

### Pogreška Gaussove eliminacije

Prema [Matrix Computations, poglavlje 3.3][GVL13], za izračunate faktore
$\hat L$ i $\hat U$ vrijedi

$$
\hat L\cdot \hat U = A+\delta A
$$

gdje je (nejednakost se čita po elementima matrica, $\varepsilon$ je sada točnost stroja)

$$
| \delta A|\leq 3(n-1) \varepsilon (|A|+|\hat L| \cdot |\hat U|) +O(\varepsilon^2).
$$

Zanemarivanje člana $O(\varepsilon^2)$ i prelazak na normu daju

$$
\|\delta A \| \approx \leq  O(n)\varepsilon (\| A\| + \| \hat L\| \cdot \| \hat U\|),
$$

pa je 

$$
 \frac{\|\delta A \|}{\|A\|} \leq O(n)\varepsilon \bigg(1+\frac{\| \hat L\| \cdot \| \hat U\|}{\|A\|}\bigg).
$$

Ukoliko se Gaussova eliminacija radi s pivotiranjem, tada će najvjerojatnije zadnji kvocijent također biti malen 
($\approx 1$). Također, pogreška kod rješavanja trokutastih sustava nije veća od navedene pa, uvrštavanjme u (1), slijedi 
da za relativnu pogrešku izračunatog rješenja vrijedi

$$
\frac{\| \hat x-x\|}{\| x\|}\leq \kappa(A) O(n\varepsilon).
$$

> __Ukoliko je kondicija matrice velika, rješenje može biti netočno.__

[GVL13]: https://books.google.hr/books?id=X5YfsuCWpxMC&printsec=frontcover&hl=hr#v=onepage&q&f=false "G. Golub and C. F Van Loan, 'Matrix Computations', 4th Edition, John Hopkins, Baltimore, 2013" 

In [62]:
n=10
v=rand(n)

10-element Array{Float64,1}:
 0.682499
 0.666193
 0.662485
 0.649548
 0.26805 
 0.778081
 0.833351
 0.457291
 0.131948
 0.410951

In [66]:
# Vandermonmde-oove matrice imaju veliku kondiciju.
A=Array{Float64}(n,n)
for i=1:n
    A[:,i]=v.^(i-1)
end
A=A'

10×10 Array{Float64,2}:
 1.0        1.0        1.0        1.0        …  1.0          1.0        
 0.682499   0.666193   0.662485   0.649548      0.131948     0.410951   
 0.465805   0.443813   0.438887   0.421912      0.0174101    0.168881   
 0.317912   0.295665   0.290756   0.274052      0.00229723   0.0694017  
 0.216975   0.19697    0.192622   0.17801       0.000303113  0.0285207  
 0.148085   0.13122    0.127609   0.115626   …  3.9995e-5    0.0117206  
 0.101068   0.0874178  0.0845391  0.0751044     5.27725e-6   0.00481659 
 0.0689789  0.0582372  0.0560059  0.0487839     6.96319e-7   0.00197938 
 0.047078   0.0387972  0.0371031  0.0316875     9.18776e-8   0.000813429
 0.0321307  0.0258464  0.0245803  0.0205825     1.2123e-8    0.00033428 

In [67]:
b=rand(n)

10-element Array{Float64,1}:
 0.52387 
 0.143127
 0.720977
 0.763094
 0.525443
 0.474195
 0.92232 
 0.598519
 0.671318
 0.457301

In [68]:
x=A\b

10-element Array{Float64,1}:
    -2.37152e9 
     2.42584e10
    -2.53373e10
     3.44215e9 
    -2.48544e5 
     1.42841e7 
    -1.71461e6 
    -9.14781e6 
 15739.6       
     5.04044e6 

In [69]:
cond(A)

5.470445700841842e11

In [70]:
Ab=Array{BigFloat}(n,n)
bb=Array{BigFloat}(n)
for i=1:n
    for j=1:n
        Ab[i,j]=map(BigFloat,A[i,j])
    end
    bb[i]=map(BigFloat,b[i])
end
xb=Ab\bb

10-element Array{BigFloat,1}:
 -2.37152281665766708938731157543310467058443507560848901733190197418477552055396e+09 
  2.425846817680698648166660960796716969638159851242582909079218660402854857721627e+10
 -2.533732542616327006233828965187865724472016193757876371622825452038537378645657e+10
  3.442150700746256138753007422934199075243663868323497764941503203463674192615493e+09
 -2.485439812861360601484347049601825703719062340160073372305414684095346598324209e+05
  1.428416370752078193407458279638283466970970673887612157452330643367118299534375e+07
 -1.714614269164924649907280791190565516392436874096244462403766187363113285052784e+06
 -9.147822561866785273260092826201621505792012183081041811584214404970085795538896e+06
  1.573963428636219435496204773735284814037763474781665661003355324466710838731533e+04
  5.040443262075458298859214115706115614378875996486444144051869151753420815515814e+06

In [71]:
norm(xb-x)/norm(xb)

1.019151668779191667952169558717566051375184570406346022553550326817793471121124e-06

### Umjetno loša kondicija

In [72]:
A=[1 1; 1 2]
b=[1;3]
x=A\b
@show x,cond(A)
A1=[1e-4 1e-4;1 2]
b1=[1e-4;3]
x1=A1\b1
x,cond(A1),x-x1

(x, cond(A)) = ([-1.0, 2.0], 6.854101966249685)


([-1.0, 2.0], 50000.00017991671, [8.88178e-16, -4.44089e-16])

## Rezidual


Izračunato rješenje $\hat x$ sustava $Ax=b$ je točno rješenje nekog sličnog sustava (vidi [Afternotes on Numerical Analysis, str. 128][Ste96]):


$$ 
(A+\delta A)\,\hat x=b. \tag{1}
$$

__Rezidual__ (ili __ostatak__) definiramo kao 

$$
r=b-A\hat x.
$$

Tada je 

$$
0=b-(A+\delta A)\,\hat x=r- \delta A\,\hat x
$$

pa je 

$$ 
\| r\| \leq \| \delta A\,\hat x \| \leq \| \delta A\| \cdot \|\hat x \|,
$$

odnosno

$$
\frac{\|  \delta A\|}{\|A \|} \geq \frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$

Dakle,

> ako  _relativni rezidual_ 
>
> $$ \frac{r}{\| A\| \cdot \|\hat x \|}$$
> 
> ima veliku normu, tada __rješenje nije izračunato stabilno.__

S druge strane, ako relativni rezidual ima malu normu, tada je rješenje izračunato stabilno. Naime, za

$$
\delta A=\frac{r\hat x^T}{\|\hat x\|^2}
$$

vrijedi (1):

$$
b-(A+\delta A)\hat x=(b-A\hat x)-\delta A \hat x = r-\frac{r\hat x^T \hat x}{\|\hat x\|^2}
= r-\frac{r \|\hat x^T \hat x\|}{\|\hat x\|^2}=r-r=0.
$$

Također vrijedi

$$
\frac{\|  \delta A\|}{\|A \|}  \leq  \frac{\|r\|\|\hat x \|}{\| A\| \cdot \|\hat x \|^2}=
\frac{\|r\|}{\| A\| \cdot \|\hat x \|}.
$$




[Ste96]: https://books.google.hr/books?id=w-2PWh01kWcC&printsec=frontcover&hl=hr#v=onepage&q&f=false    "G. W. Stewart, 'Afternotes on Numerical Analysis', SIAM, Philadelphia, 1996"

In [73]:
r=b-A*x

2-element Array{Float64,1}:
 0.0
 0.0

In [74]:
norm(r)/(norm(A)*norm(x))

0.0